In [67]:
import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns

# Human Data Science - Course Project

## Costruzione del dataset
L'obiettivo è la creazione di un dataset o più dataset contenenti dati riguardo al crimine e fattori socio-economici che possono influenzare il crimine stesso per ogni paese dell'Unione Europea.


I dati riguardo i crimini commessi sono forniti da [Eurostat](https://ec.europa.eu/eurostat/databrowser/view/crim_off_cat__custom_8028311/default/table?lang=en), [Explanatory Texts](https://ec.europa.eu/eurostat/cache/metadata/en/crim_off_cat_esms.htm)

In [68]:
df = pandas.read_table('data/crime/crim_off_cat.tsv')  #ok

display(df)

,"freq,iccs,unit,geo\TIME_PERIOD",2016,2017,2018,2019,2020,2021
0,"A,ICCS0101,P_HTHAB,AT",0.56,0.70,0.83,0.84,0.61,0.66
1,"A,ICCS0101,P_HTHAB,BE",1.55,1.74,1.68,1.27,1.26,1.26
2,"A,ICCS0101,P_HTHAB,BG",1.10,1.34,1.30,1.14,0.95,1.27
3,"A,ICCS0101,P_HTHAB,CY",1.30,0.82,1.62,1.48,1.69,1.56
4,"A,ICCS0101,P_HTHAB,CZ",0.62,0.38,0.52,0.76,0.53,0.43
...,...,...,...,...,...,...,...
103,"A,ICCS0703,P_HTHAB,PT",1.47,1.71,1.61,1.65,1.50,2.06
104,"A,ICCS0703,P_HTHAB,RO",7.09,27.67,22.42,21.07,17.66,20.92
105,"A,ICCS0703,P_HTHAB,SE",213.55,211.05,207.81,201.53,191.63,205.03
106,"A,ICCS0703,P_HTHAB,SI",56.15,47.00,10.31,10.52,14.89,11.38


In [69]:
split = df["freq,iccs,unit,geo\\TIME_PERIOD"].str.split(",", expand=True)
split.columns = ["freq", "iccs", "unit", "geo"]
split.drop(columns=["freq", "unit"], inplace=True)
split = split[["geo", "iccs"]]

df = pd.concat([df, split], axis=1)
df.drop(columns=["freq,iccs,unit,geo\\TIME_PERIOD"], inplace=True)

df.rename(columns=lambda x: x.strip(), inplace=True)

In [70]:
# Rename rows with the name of the country
geo_dict = {
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CZ": "Czechia",
    "DK": "Denmark",
    "DE": "Germany",
    "EE": "Estonia",
    "IE": "Ireland",
    "EL": "Greece",
    "ES": "Spain",
    "FR": "France",
    "HR": "Croatia",
    "IT": "Italy",
    "CY": "Cyprus",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "HU": "Hungary",
    "MT": "Malta",
    "NL": "Netherlands",
    "AT": "Austria",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "FI": "Finland",
    "SE": "Sweden",
}

df["geo"] = df["geo"].map(geo_dict)

# df = df.melt(id_vars=["geo", "iccs"], var_name="year", value_name="value")
display(df)

,2016,2017,2018,2019,2020,2021,geo,iccs
0,0.56,0.70,0.83,0.84,0.61,0.66,Austria,ICCS0101
1,1.55,1.74,1.68,1.27,1.26,1.26,Belgium,ICCS0101
2,1.10,1.34,1.30,1.14,0.95,1.27,Bulgaria,ICCS0101
3,1.30,0.82,1.62,1.48,1.69,1.56,Cyprus,ICCS0101
4,0.62,0.38,0.52,0.76,0.53,0.43,Czechia,ICCS0101
...,...,...,...,...,...,...,...,...
103,1.47,1.71,1.61,1.65,1.50,2.06,Portugal,ICCS0703
104,7.09,27.67,22.42,21.07,17.66,20.92,Romania,ICCS0703
105,213.55,211.05,207.81,201.53,191.63,205.03,Sweden,ICCS0703
106,56.15,47.00,10.31,10.52,14.89,11.38,Slovenia,ICCS0703


### Legenda
ICCS = International Classification of crime for statistical purposes
Geo = Geopolitical entity 
Unit = Unit of measure (Per 100 000 inhabitants)
Freq = Time frequency (A = Annual)

Nel dataset sono presenti record per 20 tipi di crimini diversi, purtroppo per alcuni crimini i dati erano incompleti, mancanti o completamente non presenti per alcuni paesi. 
Per questo motivo sono stati selezionati solo alcuni tipi di crimini per ogni paese:
- Intentional Homicide
- Sexual Violence
- Theft
- Corruption (Alcuni valori sono stati interpolati linearmente)

I dati sono forniti per ogni paese dell'Unione Europea, per ogni anno dal 2016 al 2021.

In [71]:
# Estrapolazione dei dati riguardo i crimini di interesse

# Intentional homicide
homicide_df = df[df["iccs"] == "ICCS0101"].drop(columns=["iccs"])
homicide_df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
homicide_df = homicide_df.melt(id_vars=["geo"], var_name="year", value_name="homicides")

# Sexual violence
sexual_violence_df = df[df["iccs"] == "ICCS0301"].drop(columns=["iccs"])
sexual_violence_df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
sexual_violence_df = sexual_violence_df.melt(id_vars=["geo"], var_name="year", value_name="sexual_violence")

# Theft
theft_df = df[df["iccs"] == "ICCS0502"].drop(columns=["iccs"])
theft_df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
theft_df = theft_df.melt(id_vars=["geo"], var_name="year", value_name="theft")

# Corruption
corruption_df = df[df["iccs"] == "ICCS0703"].drop(columns=["iccs"])
corruption_df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
corruption_df = corruption_df.melt(id_vars=["geo"], var_name="year", value_name="corruption")

In [72]:
# Interpolate linearly the data with type b (break in the series)
# 1) Sostituisce i valori contententi la stringa 'b' con None
# 2) Interpola linearmente i valori None
# 3) Resetta l'indice
def clean_interpolate(df, index):
    df.set_index(index, inplace=True)
    df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
    df = df.applymap(lambda x: None if 'b' in str(x) else x)
    df = df.interpolate(method='linear', axis=1, limit_direction='both')
    df.reset_index(inplace=True)
    return df


# Convert the data to the format: per 100.000 inhabitants
def convert_to_ht(population_df, df):
    population_df.set_index("geo", inplace=True)
    df.set_index("geo", inplace=True)

    df = df.div(100)
    df = df.mul(population_df, axis=0) / 100000

    df.reset_index(inplace=True)
    population_df.reset_index(inplace=True)

    return df

### Dataset popolazione
[Population on 1 January](https://ec.europa.eu/eurostat/databrowser/view/tps00001__custom_7848350/default/table?lang=en)
Nel presente dataset sono contenute le registrazioni della popolazione totale di ciascun paese dell'Unione Europea al 1° gennaio di ciascun anno. 
Tale dataset è stato impiegato per standardizzare i dati di tutti gli altri dataset in cui i valori erano espressi come percentuale rispetto alla popolazione totale, 
normalizzandoli al formato di 'ogni 100.000 abitanti', come i dati relativi ai crimini.

- Time: 2016-2021

### Flags
- b: break in time series
- ep: estimated, provisional
- p: provisional
- e: estimated

In [73]:
population_df = pandas.read_table('data/population/tps00001.tsv')

In [74]:
split = population_df["freq,indic_de,geo\\TIME_PERIOD"].str.split(",", expand=True)
split.drop(columns=[0, 1], inplace=True)
split.columns = ["geo"]

population_df = pd.concat([population_df, split], axis=1)
population_df.drop(columns=["freq,indic_de,geo\\TIME_PERIOD"], axis=1, inplace=True)
population_df.rename(columns=lambda x: x.strip(), inplace=True)
population_df["geo"] = population_df["geo"].map(geo_dict)

# Clean and interpolate the data
population_df = clean_interpolate(population_df, "geo")
display(population_df)

,geo,2016,2017,2018,2019,2020,2021
0,Austria,8700471.0,8772865.0,8822267.0,8858775.0,8901064.0,8932664.0
1,Belgium,11311117.0,11351727.0,11398589.0,11455519.0,11522440.0,11554767.0
2,Bulgaria,7153784.0,7101859.0,7050034.0,7000039.0,6951482.0,6916548.0
3,Cyprus,848319.0,854802.0,864236.0,875899.0,888005.0,896007.0
4,Czechia,10553843.0,10578820.0,10610055.0,10649800.0,10693939.0,10693939.0
5,Germany,82175684.0,82521653.0,82792351.0,83019213.0,83166711.0,83155031.0
6,Denmark,5707251.0,5748769.0,5781190.0,5806081.0,5822763.0,5840045.0
7,Estonia,1315944.0,1315635.0,1319133.0,1324820.0,1328976.0,1330068.0
8,Greece,10783748.0,10768193.0,10741165.0,10724599.0,10718565.0,10678632.0
9,Spain,46440099.0,46528024.0,46658447.0,46937060.0,47332614.0,47398695.0


----- Da rivedere -----
Interpolazione per i dati con parametro p, 
Cosa fare invece quando si hanno dati con flag?
- e: estimated
- p: provisional
- ep: estimated, provisional
Continuare con l'interpolazione?

----- Da rivedere -----

### Altro dataset: Persone a rischio di povertà o esclusione sociale
[People at rick of poverty or social exclusion](https://ec.europa.eu/eurostat/databrowser/view/sdg_01_10__custom_7836001/default/table?lang=en)

- Age: Less than 18 years
- Time: 2016-2021
- Sex: Total
- Unit:  percentage 

In [75]:
poverty_df = pandas.read_table('data/sdg/sdg_01_10.tsv')
display(poverty_df)

,"freq,sex,age,unit,geo\TIME_PERIOD",2016,2017,2018,2019,2020,2021
0,"A,T,Y_LT18,PC,AT",20.5,22.7,21.6,20.1,21.9,22.8
1,"A,T,Y_LT18,PC,BE",23.5,24.2,23.5,23.0 b,22.0,20.5
2,"A,T,Y_LT18,PC,BG",47.3 b,42.4,33.5,35.8,35.5,33.0
3,"A,T,Y_LT18,PC,CY",23.2,21.7,20.5,20.3,19.0,19.2
4,"A,T,Y_LT18,PC,CZ",16.4,14.3,13.0,13.0,12.9,13.3
5,"A,T,Y_LT18,PC,DE",20.6,18.6,17.5,15.4,22.3 b,23.7
6,"A,T,Y_LT18,PC,DK",14.0,15.1,15.4,13.8,13.4 b,14.0
7,"A,T,Y_LT18,PC,EE",19.7,18.8,17.0,19.7,17.4,17.4
8,"A,T,Y_LT18,PC,EL",37.2,36.5,34.1,31.2,30.8,32.0
9,"A,T,Y_LT18,PC,ES",33.7,31.9,30.5,31.0,31.6,33.4


In [76]:
split = poverty_df["freq,sex,age,unit,geo\\TIME_PERIOD"].str.split(",", expand=True)
split.drop(columns=[0, 1, 2, 3], inplace=True)
split.columns = ["geo"]

poverty_df = pd.concat([poverty_df, split], axis=1)
poverty_df.drop(columns=["freq,sex,age,unit,geo\\TIME_PERIOD"], axis=1, inplace=True)
poverty_df.rename(columns=lambda x: x.strip(), inplace=True)
poverty_df["geo"] = poverty_df["geo"].map(geo_dict)

Interopolazione lineare per i dati di tipo b (break in the series)

In [77]:
poverty_df = clean_interpolate(poverty_df, "geo")

display(poverty_df)

,geo,2016,2017,2018,2019,2020,2021
0,Austria,20.5,22.7,21.6,20.10,21.90,22.8
1,Belgium,23.5,24.2,23.5,22.75,22.00,20.5
2,Bulgaria,42.4,42.4,33.5,35.80,35.50,33.0
3,Cyprus,23.2,21.7,20.5,20.30,19.00,19.2
4,Czechia,16.4,14.3,13.0,13.00,12.90,13.3
5,Germany,20.6,18.6,17.5,15.40,19.55,23.7
6,Denmark,14.0,15.1,15.4,13.80,13.90,14.0
7,Estonia,19.7,18.8,17.0,19.70,17.40,17.4
8,Greece,37.2,36.5,34.1,31.20,30.80,32.0
9,Spain,33.7,31.9,30.5,31.00,31.60,33.4


I dati sono stati interpolati linearmente, ma non è detto che sia la scelta migliore.

I dati sono fortini in percentuale alla popolazione locale del paese. È necessaraio effettuare una normalizzazione per avere un confronto tra i paesi, portando quindi tutti i dati nel formato: ogni 100.000 abitanti.

In [78]:
poverty_df = convert_to_ht(population_df, poverty_df)

poverty_df = poverty_df.melt(id_vars=["geo"], var_name="year", value_name="poverty")

display(poverty_df)

,geo,year,poverty
0,Austria,2016,17.835966
1,Belgium,2016,26.581125
2,Bulgaria,2016,30.332044
3,Cyprus,2016,1.968100
4,Czechia,2016,17.308303
...,...,...,...
157,Portugal,2021,23.582997
158,Romania,2021,80.070931
159,Sweden,2021,20.447211
160,Slovenia,2021,2.319875


### Altro dataset: NEET, Not in Education, Employment or Training

[Young people neither in employment nor in education and training by sex (NEET)](https://ec.europa.eu/eurostat/databrowser/view/sdg_08_20__custom_7848499/default/table?lang=en)

- Age: 15-29 years
- Time: 2016-2021
- Unit: percentage

In [79]:
neet_df = pandas.read_table('data/sdg/sdg_08_20.tsv')

#display(neet_df)

In [80]:
split = neet_df["freq,sex,age,unit,geo\\TIME_PERIOD"].str.split(",", expand=True)
split.drop(columns=[0, 1, 2, 3], inplace=True)
split.columns = ["geo"]

neet_df = pd.concat([neet_df, split], axis=1)
neet_df.drop(columns=["freq,sex,age,unit,geo\\TIME_PERIOD"], axis=1, inplace=True)
neet_df = neet_df.iloc[:, ::-1]
neet_df.rename(columns=lambda x: x.strip(), inplace=True)
neet_df["geo"] = neet_df["geo"].map(geo_dict)

# Clean and interpolate the data
neet_df = clean_interpolate(neet_df, "geo")
# Convert to per 100.000 inhabitants
neet_df = convert_to_ht(population_df, neet_df)

neet_df = neet_df.melt(id_vars=["geo"], var_name="year", value_name="neet")
display(neet_df)

,geo,year,neet
0,Austria,2016,8.091438
1,Belgium,2016,13.799563
2,Bulgaria,2016,16.024476
3,Cyprus,2016,1.526974
4,Czechia,2016,11.714766
...,...,...,...
157,Portugal,2021,9.783339
158,Romania,2021,38.979374
159,Sweden,2021,6.227577
160,Slovenia,2021,1.539553


### Altro dataset: Tertiary educational attainment by sex

[Tertiary educational attainment by sex](https://ec.europa.eu/eurostat/databrowser/view/sdg_04_20__custom_7957119/default/table?lang=en)

Legenda:
- Age: 25-34 years
- Unit: percentage 
- Time frequency: Annual

In [81]:
education_df = pandas.read_table('data/sdg/sdg_04_20.tsv')

#display(education_df)

In [82]:
split = education_df["freq,sex,age,unit,isced11,geo\\TIME_PERIOD"].str.split(",", expand=True)
split.drop(columns=[0, 1, 2, 3, 4], inplace=True)
split.columns = ["geo"]

education_df = pd.concat([education_df, split], axis=1)
education_df.drop(columns=["freq,sex,age,unit,isced11,geo\\TIME_PERIOD"], axis=1, inplace=True)
education_df.rename(columns=lambda x: x.strip(), inplace=True)
education_df["geo"] = education_df["geo"].map(geo_dict)

# Clean and interpolate the data
education_df = clean_interpolate(education_df, "geo")
# Convert to per 100.000 inhabitants
#education_df = convert_to_ht(population_df, education_df)

education_df = education_df.melt(id_vars=["geo"], var_name="year", value_name="education")
display(education_df)

,geo,year,education
0,Austria,2016,39.7
1,Belgium,2016,44.3
2,Bulgaria,2016,32.8
3,Cyprus,2016,56.2
4,Czechia,2016,32.6
...,...,...,...
157,Portugal,2021,41.9
158,Romania,2021,24.9
159,Sweden,2021,49.2
160,Slovenia,2021,45.4


### Altro dataset: Income quintile share ratio (S80/S50)
[Income quintile share ratio (S80/S20)](https://ec.europa.eu/eurostat/databrowser/view/ilc_di11__custom_7950467/default/table?lang=en)

[Explenatory texts](https://ec.europa.eu/eurostat/cache/metadata/en/ilc_sieusilc.htm)
The data are flagged when it is needed. The following flags are used:
 (b)  Break in series (i.e. change of source or change of methodology from that used in preceding year); !
 
 Indicatore statistico che viene spesso utilizzato nell'analisi economica e sociale per misurare la distribuzione del reddito o del reddito disponibile tra diversi gruppi demografici all'interno di una popolazione. 
 In questo caso il dataset descrive il rapporto calcolato tra il quintile più ricco (S80) e il quintile mediano (S50). 
 Questo significa che si sta confrontando il 20% più ricco della popolazione con il 50% mediano, il che offre una visione della disuguaglianza tra i redditi più alti e il reddito mediano.


In [83]:
inequity_df = pandas.read_table('data/population/ilc_di11.tsv')

split = inequity_df["freq,age,sex,unit,geo\\TIME_PERIOD"].str.split(",", expand=True)
split.drop(columns=[0, 1, 2, 3], inplace=True)
split.columns = ["geo"]

inequity_df = pd.concat([inequity_df, split], axis=1)
inequity_df.drop(columns=["freq,age,sex,unit,geo\\TIME_PERIOD"], axis=1, inplace=True)
inequity_df.rename(columns=lambda x: x.strip(), inplace=True)
inequity_df["geo"] = inequity_df["geo"].map(geo_dict)

# Clean and interpolate the data
inequity_df = clean_interpolate(inequity_df, "geo") # Non necessario !

inequity_df = inequity_df.melt(id_vars=["geo"], var_name="year", value_name="inequity")
display(inequity_df)

,geo,year,inequity
0,Austria,2016,4.09
1,Belgium,2016,3.85
2,Bulgaria,2016,8.23
3,Cyprus,2016,4.88
4,Czechia,2016,3.50
...,...,...,...
157,Portugal,2021,5.66
158,Romania,2021,7.14
159,Sweden,2021,4.04
160,Slovenia,2021,3.24


 
### Altro dataset: Median income by age and sex
Da vedere
[link](https://ec.europa.eu/eurostat/databrowser/view/ilc_di03__custom_7988486/default/table?lang=en)

In [84]:
income_df = pandas.read_table('data/population/ilc_di03.tsv')

split = income_df["freq,age,sex,indic_il,unit,geo\\TIME_PERIOD"].str.split(",", expand=True)
split.drop(columns=[0, 1, 2, 3, 4], inplace=True)
split.columns = ["geo"]

income_df = pd.concat([income_df, split], axis=1)
income_df.drop(columns=["freq,age,sex,indic_il,unit,geo\\TIME_PERIOD"], axis=1, inplace=True)
income_df.rename(columns=lambda x: x.strip(), inplace=True)
income_df["geo"] = income_df["geo"].map(geo_dict)

# Clean and interpolate the data
income_df = clean_interpolate(income_df, "geo")

income_df = income_df.melt(id_vars=["geo"], var_name="year", value_name="income")
display(income_df)

,geo,year,income
0,Austria,2016,24400.0
1,Belgium,2016,22267.0
2,Bulgaria,2016,3649.0
3,Cyprus,2016,14132.0
4,Czechia,2016,7905.0
...,...,...,...
157,Portugal,2021,11115.0
158,Romania,2021,4914.0
159,Sweden,2021,25925.0
160,Slovenia,2021,15415.0


# Costruzione del o dei dataset finali per l'analisi
### Dataset Generale

In [85]:
# Merge dei dataset

theft_df = pd.merge(theft_df, poverty_df, on=["geo", "year"])
theft_df = pd.merge(theft_df, neet_df, on=["geo", "year"])
theft_df = pd.merge(theft_df, education_df, on=["geo", "year"])
theft_df = pd.merge(theft_df, income_df, on=["geo", "year"])
theft_df = pd.merge(theft_df, inequity_df, on=["geo", "year"])

theft_df.to_csv("data/theft_df.csv", index=False)

display(theft_df)

,geo,year,theft,poverty,neet,education,income,inequity
0,Austria,2016,1651.67,17.835966,8.091438,39.7,24400.0,4.09
1,Belgium,2016,2055.54,26.581125,13.799563,44.3,22267.0,3.85
2,Bulgaria,2016,457.48,30.332044,16.024476,32.8,3649.0,8.23
3,Cyprus,2016,86.17,1.968100,1.526974,56.2,14132.0,4.88
4,Czechia,2016,674.86,17.308303,11.714766,32.6,7905.0,3.50
...,...,...,...,...,...,...,...,...
157,Portugal,2021,621.02,23.582997,9.783339,41.9,11115.0,5.66
158,Romania,2021,471.44,80.070931,38.979374,24.9,4914.0,7.14
159,Sweden,2021,2784.26,20.447211,6.227577,49.2,25925.0,4.04
160,Slovenia,2021,634.76,2.319875,1.539553,45.4,15415.0,3.24
